# Control of platform angle and wheel velocity with gravity

Do all imports:

In [ ]:
# Stuff for computation
import numpy as np
import sympy as sym
from scipy import linalg

# Stuff for visualization
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import PreText

[Display Bokeh plots inline](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html#classic-notebooks):

In [ ]:
output_notebook()

Suppress the use of scientific notation when printing small numbers:

In [ ]:
np.set_printoptions(suppress=True)

The equations of motion are

$$
\begin{aligned}
J_1 \ddot{q}_1 &= \tau - m_wgl\sin(q_1) \\
J_2 \dot{v}_2 &= - \left( \dfrac{J_1 + J_2}{J_1} \right) \tau + \left( \dfrac{J_2}{J_1} \right) m_wgl\sin(q_1)
\end{aligned}
$$

where

* $q_1$ is the platform angle
* $\dot{q}_1$ is the platform velocity
* $v_2$ is the wheel velocity
* $\tau$ is the torque applied *to* the platform *by* the wheel

and

$$
J_1 = J_p + m_wl^2
\qquad
\qquad
J_2 = J_w
$$

where parameters are as follows:

In [ ]:
# distance from platform axis to wheel axis
l = 1.

# radius of wheel
rw = 0.5

# mass of wheel
mw = 0.25

# moi of wheel
Jw = 0.5 * mw * rw**2

# mass of platform
mp = 12. * (1. - mw * l**2) / (3.**2 + 2.**2)

# moment of inertia of platform
Jp = (1. / 12.) * mp * (3.**2 + 2.**2)

# gravity
g = 9.81

# composite parameters
J1 = Jp + mw * l**2
J2 = Jw

Define a function to linearize these equations of motion about some choice of equilibrium point and return a state-space model:

In [ ]:
# Define a function to return a state-space model for a given choice of equilibrium point.
def get_model(q1e, v1e, v2e, taue):
    # Define variables
    q1, v1, v2, tau = sym.symbols('q1, v1, v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1,
        (1 / J1) * (tau - mw * g * l * sym.sin(q1)),
        (1 / J2) * (- ((J1 + J2) / J1) * tau + (J2 / J1) * mw * g * l * sym.sin(q1))
    ])

    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, v2])
    B_sym = f.jacobian([tau])

    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, v2, tau], B_sym)

    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, v1e, v2e, taue).astype(float)

    # Return A and B
    return A, B

Visualize the result of applying linear state feedback:

In [ ]:
(q1e, v1e, v2e, taue) = (np.pi, 0., 0., 0.) # <-- EQUILIBRIUM POINT
A, B = get_model(q1e, v1e, v2e, taue)
(t0, t1, dt) = (0., 5., 0.05)
nt = int(1 + np.ceil((t1 - t0) / dt))
x0 = np.array([1., 0., 0.])
t = np.linspace(t0, t1, nt)
x = np.zeros((101, 3))

def update(k1=0, k2=0, k3=0):
    # Get closed-loop eigenvalues
    K = np.array([[k1, k2, k3]])
    s = linalg.eigvals(A - B @ K)
    s_plt.data_source.data['x'] = s.real
    s_plt.data_source.data['y'] = s.imag
    eigs.text = 'EIGENVALUES:\n' \
             + f' s1 = {s[0]:12.2f}\n' \
             + f' s2 = {s[1]:12.2f}\n' \
             + f' s3 = {s[2]:12.2f}'
    code.text = 'CONTROLLER:\n' \
             + f' q1 = platform_angle\n' \
             + f' v1 = platform_velocity\n' \
             + f' v2 = wheel_velocity\n' \
             + f' wheel_torque = - ({k1:g}) * (q1 - {q1e:.2f})' \
             + f' - ({k2:g}) * (v1 - {v1e:.2f})' \
             + f' - ({k3:g}) * (v2 - {v2e:.2f})' \
    
    # Get x(t)
    for i in range(len(t)):
        x[i] = linalg.expm((A - B @ K) * (t[i] - t0)) @ x0
    x1_plt.data_source.data['y'] = x[:, 0]
    x2_plt.data_source.data['y'] = x[:, 1]
    x3_plt.data_source.data['y'] = x[:, 2]
    
    # Refresh plots
    push_notebook()

s_fig = figure(title='closed-loop eigenvalues', height=300, width=300,
                x_range=(-15, 5), y_range=(-10, 10))
s_plt = s_fig.circle([0, 0, 0], [0, 0, 0], size=5, color='navy', alpha=0.5)
x1_fig = figure(title='x1(t)', height=150, width=400,
                x_range=(t0, t1), y_range=(-1.5, 1.5))
x1_plt = x1_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
x2_fig = figure(title='x2(t)', height=150, width=400,
                x_range=(t0, t1), y_range=(-5, 5))
x2_plt = x2_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
x3_fig = figure(title='x3(t)', height=150, width=400,
                x_range=(t0, t1), y_range=(-200, 200))
x3_plt = x3_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
eigs = PreText(text='', width=300, height=100, margin=(50, 0, 0, 50))
code = PreText(text='', margin=(20, 0, 0, 20))

show(
    column(
        row(
            column(s_fig, eigs),
            column(x1_fig, x2_fig, x3_fig)
        ),
        row(code, sizing_mode='stretch_width'),
    ),
    notebook_handle=True,
)
interact(update, k1=(-50, 150), k2=(-25, 75), k3=(-2, 3, 0.01));